# Level 2 - Week 3 - 02 Filters and Top-K Tradeoffs

**Estimated time:** 60-90 minutes

## Learning Objectives

- Compare precision and recall tradeoffs
- Understand filter impact
- Log retrieval settings


## Overview

Filters improve precision, `top_k` improves recall.

### Underlying theory: precision@k and recall@k

Let:

- $R$ be the set of relevant chunk ids for a query
- $L_k = [\ell_1, \ldots, \ell_k]$ be the ordered list of retrieved chunk ids
- $\text{hits}_k = R \cap \{\ell_1, \ldots, \ell_k\}$

Then:

$$
\mathrm{Precision@k} = \frac{|\text{hits}_k|}{k}
$$

$$
\mathrm{Recall@k} = \frac{|\text{hits}_k|}{|R|}
$$

Intuition:

- Precision@k is “how noisy are the retrieved chunks?”
- Recall@k is “did I get the evidence I needed at all?”

### Filters as restricting the search space

A filter restricts the candidate set from “all chunks” to “chunks matching this predicate”.

- correct filters can remove irrelevant regions and improve precision
- wrong filters can remove all relevant chunks and collapse recall to 0

## Practice Steps

- Compute precision and recall for toy results.
- Compare two `top_k` settings.
- Change retrieved lists to simulate different filter outcomes.

### Sample code

Toy precision and recall calculation.


In [ ]:
def precision_at_k(relevant: set[str], retrieved: list[str]) -> float:
    if not retrieved:
        return 0.0
    hits = [r for r in retrieved if r in relevant]
    return len(hits) / len(retrieved)


def recall_at_k(relevant: set[str], retrieved: list[str]) -> float:
    if not relevant:
        return 0.0
    hits = [r for r in retrieved if r in relevant]
    return len(hits) / len(relevant)

relevant = {'A', 'B', 'C', 'D'}
retrieved_3 = ['A', 'B', 'X']
retrieved_8 = ['A', 'B', 'X', 'Y', 'C', 'Z', 'W', 'V']

print('p@3', precision_at_k(relevant, retrieved_3))
print('r@3', recall_at_k(relevant, retrieved_3))
print('p@8', precision_at_k(relevant, retrieved_8))
print('r@8', recall_at_k(relevant, retrieved_8))


### Student fill-in

Try a different filter or top_k and record the tradeoff.


In [ ]:
def precision_at_k(relevant: set[str], retrieved: list[str], k: int) -> float:
    topk = retrieved[:k]
    if k <= 0:
        raise ValueError("k must be > 0")
    if not topk:
        return 0.0
    hits = [r for r in topk if r in relevant]
    return len(hits) / k


def recall_at_k(relevant: set[str], retrieved: list[str], k: int) -> float:
    topk = retrieved[:k]
    if not relevant:
        return 0.0
    hits = [r for r in topk if r in relevant]
    return len(hits) / len(relevant)


def summarize_case(name: str, relevant: set[str], retrieved: list[str], k: int) -> None:
    p = precision_at_k(relevant, retrieved, k)
    r = recall_at_k(relevant, retrieved, k)
    hits = [cid for cid in retrieved[:k] if cid in relevant]
    print(name)
    print("  k=", k)
    print("  retrieved=", retrieved[:k])
    print("  hits=", hits)
    print("  precision@k=", round(p, 3), "recall@k=", round(r, 3))
    print("---")


# Relevant chunks for this query (ground truth)
relevant = {"A", "B", "C", "D"}

# Case A: correct filter, small top_k => higher precision, lower recall
retrieved_case_a = ["A", "B", "X"]

# Case B: correct filter, larger top_k => recall improves, precision drops
retrieved_case_b = ["A", "B", "X", "Y", "C", "Z", "W", "V"]

# Case C: wrong filter => retrieved from irrelevant subset => recall collapses
retrieved_case_c = ["P", "Q", "R", "S", "T", "U"]

summarize_case("Case A (correct filter, k=3)", relevant, retrieved_case_a, k=3)
summarize_case("Case B (correct filter, k=8)", relevant, retrieved_case_b, k=8)
summarize_case("Case C (wrong filter, k=3)", relevant, retrieved_case_c, k=3)

# Student prompt:
# - Modify retrieved_case_a/b/c to simulate your own scenarios
# - Add a case where top_k is high but only 1 relevant chunk appears early (good MRR but low recall)
# - Write down what you'd log per request: top_k, filters, returned chunk_ids+scores

## Self-check

- Do you log filters and top_k per request?
- Can you explain precision vs recall tradeoffs?
